<a href="https://colab.research.google.com/github/R4hul2002/Battery-soc/blob/main/Copy_of_Caterpillar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

newdf = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data.xlsx')

In [5]:
df = newdf.iloc[:, 3:11] #taking the necessary data columns
df

,record_create_timestamp,state_of_charge,energy_consumption,battery_lifetime,max_cell_temperature,min_cell_temperature,battery_charger_connected_lifetime
0,2023-05-01 00:08:00,91.7,4182300,2687.709,23.875,22.781,1126.533
1,2023-05-01 00:19:00,91.4,4182300,2687.881,23.875,22.781,1126.533
2,2023-05-01 00:29:00,91.3,4182300,2688.053,23.844,22.750,1126.533
3,2023-05-01 00:39:00,91.2,4182300,2688.225,23.813,22.719,1126.533
4,2023-05-01 00:50:00,91.2,4182300,2688.398,23.781,22.688,1126.533
...,...,...,...,...,...,...,...
3820,2023-05-31 18:21:00,68.0,5231000,3130.669,25.188,23.281,1265.785
3821,2023-05-31 18:31:00,67.8,5231100,3130.835,25.313,23.438,1265.785
3822,2023-05-31 18:42:00,67.5,5231200,3131.000,25.438,23.594,1265.785
3823,2023-05-31 20:46:00,67.1,5231200,3131.005,26.188,25.156,1265.785


In [6]:
df['record_create_timestamp'] = pd.to_datetime(df['record_create_timestamp'], format='%Y-%m-%dT%H:%M')

df.drop_duplicates(subset='record_create_timestamp', inplace=True)

# creating a new DataFrame with timestamps at 1-minute intervals
start_time = df['record_create_timestamp'].min()  # considering first value as starting timestamp
end_time = df['record_create_timestamp'].max()  # considering last value as ending timestamp
time_range = pd.date_range(start=start_time, end=end_time, freq='1min')
new_df = pd.DataFrame({'record_create_timestamp': time_range})

# Merging the new DataFrame with the original DataFrame based on the 'record_create_timestamp' column
merged_df = pd.merge(df, new_df, on='record_create_timestamp', how='right')

In [7]:
merged_df

,record_create_timestamp,state_of_charge,energy_consumption,battery_lifetime,max_cell_temperature,min_cell_temperature,battery_charger_connected_lifetime
0,2023-05-01 00:08:00,91.7,4182300.0,2687.709,23.875,22.781,1126.533
1,2023-05-01 00:09:00,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-05-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-05-01 00:11:00,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-05-01 00:12:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
44561,2023-05-31 22:49:00,NaN,NaN,NaN,NaN,NaN,NaN
44562,2023-05-31 22:50:00,NaN,NaN,NaN,NaN,NaN,NaN
44563,2023-05-31 22:51:00,NaN,NaN,NaN,NaN,NaN,NaN
44564,2023-05-31 22:52:00,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#interpolating the columns to fill missing values
columns_to_interpolate = ['state_of_charge', 'energy_consumption', 'battery_lifetime', 'max_cell_temperature', 'min_cell_temperature', 'battery_charger_connected_lifetime' ]
merged_df[columns_to_interpolate] = merged_df[columns_to_interpolate].interpolate()

In [ ]:
merged_df

,record_create_timestamp,state_of_charge,energy_consumption,battery_lifetime,max_cell_temperature,min_cell_temperature,battery_charger_connected_lifetime
0,2023-05-01 00:08:47,91.700000,4182300.0,2687.709000,23.875,22.781,1126.533
1,2023-05-01 00:09:47,91.683871,4182300.0,2687.725645,23.873,22.779,1126.533
2,2023-05-01 00:10:47,91.667742,4182300.0,2687.742290,23.871,22.777,1126.533
3,2023-05-01 00:11:47,91.651613,4182300.0,2687.758935,23.869,22.775,1126.533
4,2023-05-01 00:12:47,91.635484,4182300.0,2687.775581,23.867,22.773,1126.533
...,...,...,...,...,...,...,...
44560,2023-05-31 22:48:47,70.400000,5230300.0,3124.428000,22.875,21.281,1265.785
44561,2023-05-31 22:49:47,70.400000,5230300.0,3124.428000,22.875,21.281,1265.785
44562,2023-05-31 22:50:47,70.400000,5230300.0,3124.428000,22.875,21.281,1265.785
44563,2023-05-31 22:51:47,70.400000,5230300.0,3124.428000,22.875,21.281,1265.785


In [ ]:
# Save DataFrame as an Excel file
merged_df.to_excel('result.xlsx', index=False)

In [8]:
merged_df.to_csv('output.csv', index=False)